#IMPORT LIBRERIE

In [17]:
import sys
sys.path.append('./')

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [19]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install scikit-plot

from collections import defaultdict

from sklearn.model_selection import cross_val_score

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preparation

lettura e stampa del training set

In [21]:
dataset="/content/drive/My Drive/Machine Learning Project/monks/monks-2_train.csv"
df_train = pd.DataFrame(pd.read_csv(dataset))
df_train.head()

,classe,a1,a2,a3,a4,a5,a6
0,0,1,1,1,1,2,2
1,0,1,1,1,1,4,1
2,0,1,1,1,2,1,1
3,0,1,1,1,2,1,2
4,0,1,1,1,2,2,1


In [22]:
# label encoding the data
from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
  
df_train['a1']= le.fit_transform(df_train['a1'])
df_train['a2']= le.fit_transform(df_train['a2'])
df_train['a3']= le.fit_transform(df_train['a3'])
df_train['a4']= le.fit_transform(df_train['a4'])
df_train['a5']= le.fit_transform(df_train['a5'])
df_train['a6']= le.fit_transform(df_train['a6'])

y2 = pd.get_dummies(df_train.a1, prefix='a1')
y3 = pd.get_dummies(df_train.a2, prefix='a2')
y4 = pd.get_dummies(df_train.a3, prefix='a3')
y5 = pd.get_dummies(df_train.a4, prefix='a4')
y6 = pd.get_dummies(df_train.a5, prefix='a5')
y7 = pd.get_dummies(df_train.a6, prefix='a6')

df_train = df_train.drop("a1", axis=1)
df_train = df_train.drop("a2", axis=1)
df_train = df_train.drop("a3", axis=1)
df_train = df_train.drop("a4", axis=1)
df_train = df_train.drop("a5", axis=1)
df_train = df_train.drop("a6", axis=1)


frames = [df_train,y2,y3,y4,y5,y6,y7]

df_train = pd.concat(frames, axis=1, join='inner')
display(df_train)

,classe,a1_0,a1_1,a1_2,a2_0,a2_1,a2_2,a3_0,a3_1,a4_0,a4_1,a4_2,a5_0,a5_1,a5_2,a5_3,a6_0,a6_1
0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1
1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,1,0
2,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0
3,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1
4,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,1,0
165,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1
166,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,1,0
167,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0


lettura e stampa del test set

In [23]:
dataset="/content/drive/My Drive/Machine Learning Project/monks/monks-2_test.csv"
df_test = pd.DataFrame(pd.read_csv(dataset))
df_test.head()

,classe,a1,a2,a3,a4,a5,a6
0,0,1,1,1,1,1,1
1,0,1,1,1,1,1,2
2,0,1,1,1,1,2,1
3,0,1,1,1,1,2,2
4,0,1,1,1,1,3,1


In [24]:
# label encoding the data
from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
  
df_test['a1']= le.fit_transform(df_test['a1'])
df_test['a2']= le.fit_transform(df_test['a2'])
df_test['a3']= le.fit_transform(df_test['a3'])
df_test['a4']= le.fit_transform(df_test['a4'])
df_test['a5']= le.fit_transform(df_test['a5'])
df_test['a6']= le.fit_transform(df_test['a6'])

y2 = pd.get_dummies(df_test.a1, prefix='a1')
y3 = pd.get_dummies(df_test.a2, prefix='a2')
y4 = pd.get_dummies(df_test.a3, prefix='a3')
y5 = pd.get_dummies(df_test.a4, prefix='a4')
y6 = pd.get_dummies(df_test.a5, prefix='a5')
y7 = pd.get_dummies(df_test.a6, prefix='a6')

df_test = df_test.drop("a1", axis=1)
df_test = df_test.drop("a2", axis=1)
df_test = df_test.drop("a3", axis=1)
df_test = df_test.drop("a4", axis=1)
df_test = df_test.drop("a5", axis=1)
df_test = df_test.drop("a6", axis=1)


frames = [df_test,y2,y3,y4,y5,y6,y7]

df_test = pd.concat(frames, axis=1, join='inner')
display(df_test)

,classe,a1_0,a1_1,a1_2,a2_0,a2_1,a2_2,a3_0,a3_1,a4_0,a4_1,a4_2,a5_0,a5_1,a5_2,a5_3,a6_0,a6_1
0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0
1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1
2,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0
3,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1
4,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1
428,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
429,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1
430,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0


# Data Partitioning

divisione in training set e test set

In [25]:
class_name = 'classe'

attributes = [col for col in df_train.columns if col != class_name]
X_train = df_train[attributes].values
y_train = df_train[class_name]

attributes = [col for col in df_test.columns if col != class_name]
X_test = df_test[attributes].values
y_test = df_test[class_name]

faccio grid search per trovare configurazione migliore

In [26]:
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV

nfolds=10


param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 
              'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid']} 


grid = GridSearchCV(svm.SVC(), param_grid, cv=nfolds, verbose=1)

grid.fit(X_train, y_train)

print(grid.best_estimator_)


Fitting 10 folds for each of 256 candidates, totalling 2560 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


[Parallel(n_jobs=1)]: Done 2560 out of 2560 | elapsed:   12.3s finished


In [27]:
from sklearn.metrics import classification_report, confusion_matrix  

y_pred = grid.predict(X_train)

print('Accuracy %s' % accuracy_score(y_train, y_pred))
print('F1-score %s' % f1_score(y_train, y_pred, average=None))
print(classification_report(y_train, y_pred))

Accuracy 1.0
F1-score [1. 1.]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       105
           1       1.00      1.00      1.00        64

    accuracy                           1.00       169
   macro avg       1.00      1.00      1.00       169
weighted avg       1.00      1.00      1.00       169



In [28]:
from sklearn.metrics import classification_report, confusion_matrix  

y_pred = grid.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.8148148148148148
F1-score [0.85454545 0.74522293]
              precision    recall  f1-score   support

           0       0.90      0.81      0.85       290
           1       0.68      0.82      0.75       142

    accuracy                           0.81       432
   macro avg       0.79      0.82      0.80       432
weighted avg       0.83      0.81      0.82       432

